In [1]:
from transformers import GPT2LMHeadModel,AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")  # QwQ-32B
# qwq = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-7B-Instruct",device_map="auto")
qwq = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-7B-Instruct", device_map="auto", trust_remote_code=True)


/home/uestc_zhou/anaconda3/envs/llm-graph/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.95s/it]


In [6]:
import pandas as pd

import os

def merge_and_shuffle_datasets(path1, path2, output_path):
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path2)
    df_full = pd.concat([df1, df2], ignore_index=True)

    # 清除列名前后空格
    df_full.columns = df_full.columns.str.strip()

    # 打乱数据（设置随机种子以保证可复现）
    # df_full = df_full.sample(frac=1, random_state=42).reset_index(drop=True)

    df_full.to_csv(output_path, index=False)
    print(f"✅ 合并并打乱后的数据已保存至：{output_path}")
    

# merge_and_shuffle_datasets(
#     path1="../datasets/DoHBrw/raw/l2-malicious.csv",
#     path2="../datasets/DoHBrw/raw/l2-benign.csv",
#     output_path="../datasets/DoHBrw/raw/dataset.csv"
# )

merge_and_shuffle_datasets(
    path1="../datasets/CICIDS/raw/dataset_1.csv",
    path2="../datasets/CICIDS/raw/dataset_2.csv",
    output_path="../datasets/CICIDS/raw/dataset.csv"
)


✅ 合并并打乱后的数据已保存至：../datasets/CICIDS/raw/dataset.csv


In [3]:
file_list = ['Network_dataset_' + str(i+1) + '.csv' for i in range(23)]
data_dir = '../datasets/TONIoT/raw/'
# 初始化空DataFrame
df_all = pd.DataFrame()

for fname in file_list:
    path = os.path.join(data_dir, fname)
    df = pd.read_csv(path)
    df_all = pd.concat([df_all, df], ignore_index=True)

print("Total samples:", len(df_all))

/home/uestc_zhou/tmp/ipykernel_691/146980376.py:8: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
/home/uestc_zhou/tmp/ipykernel_691/146980376.py:8: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
/home/uestc_zhou/tmp/ipykernel_691/146980376.py:8: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Total samples: 22339022


In [5]:
df_all.to_csv('../datasets/TONIoT/raw/dataset.csv', index=False)

In [4]:
import openai

def generate_description(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="Qwen2-7B",
            messages=[
                {"role": "system", "content": "你是一个网络安全专家"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=150
        )
        return response["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"LLM 调用失败：{e}")
        return "描述生成失败"
    


In [ ]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
from features import load_and_clean_data, extract_key_features
from llm_generator import build_prompt
from llm_generator import generate_description
from embedder import encode_descriptions
import pandas as pd
import numpy as np


DATA_PATH = "data/dataset.csv"
OUT_DESC = "outputs/descriptions.csv"
OUT_EMB = "outputs/embeddings.npy"

# 确保数据文件夹存在
os.makedirs(os.path.dirname(DATA_PATH), exist_ok=True)

df = load_and_clean_data(DATA_PATH)
descriptions = []

print("开始生成流量描述文本...")
for _, row in df.iterrows():
    features = extract_key_features(row)
    prompt = build_prompt(features)
    description = generate_description(prompt)
    descriptions.append(description)
    print(f"描述：{description}")

df["description"] = descriptions

# 确保输出文件夹存在
os.makedirs("outputs", exist_ok=True)
df.to_csv(OUT_DESC, index=False)

print("开始编码描述向量...")
embeddings = encode_descriptions(descriptions)
np.save(OUT_EMB, embeddings)
print("编码完成，保存至:", OUT_EMB)

开始生成流量描述文本...


KeyError: 'c_ack_cnt:4'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer, GenerationConfig
import threading

from llm_generator import build_prompt

# 构建对话消息并应用 Chat 模板
prompt = build_prompt()

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to(qwq.device)

# 创建 Text Streamer
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# 配置生成参数
gen_config = GenerationConfig(
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    # eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

# 使用线程启动生成（非阻塞）
thread = threading.Thread(target=qwq.generate, kwargs={
    "inputs": inputs["input_ids"],
    "generation_config": gen_config,
    "streamer": streamer
})
thread.start()

# 主线程中实时输出生成的文本
print("\n" + "=" * 20 + "streaming response" + "=" * 20 + "\n")
for new_text in streamer:
    print(new_text, end="", flush=True)


====================streaming response====================

在数值比较中，数字越大表示的值越高。因此，将 3.11 和 3.8 进行比较：

- 3.11 的数值小于 3.8。

所以，3.8 比 3.11 大。

In [6]:
streamer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-7B-Instruct", device_map="auto", trust_remote_code=True)

# 构造聊天 prompt
messages = [
    {"role": "system", "content": "你是一个乐于助人的 AI 助手"},
    {"role": "user", "content": "你好，请介绍一下你自己"},
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(text, return_tensors="pt").to(model.device)
output = model.generate(**inputs, max_new_tokens=256)

response = tokenizer.decode(output[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
print("🤖:", response)

In [3]:
import numpy as np

data = np.load("outputs/embeddings_10.npy")


array([[-0.12450424,  0.03015982, -0.0465565 , ...,  0.04519766,
         0.03851491,  0.03977296],
       [-0.12822382,  0.05868031, -0.05762715, ...,  0.03317084,
         0.03382321,  0.04326383],
       [-0.07602066,  0.01724214,  0.00339649, ...,  0.02214848,
         0.06944926,  0.0597745 ],
       ...,
       [-0.08526272,  0.05421873, -0.02398501, ...,  0.04866201,
         0.0733339 ,  0.06499132],
       [-0.13573076,  0.01337384, -0.01690241, ...,  0.01925463,
         0.0136504 ,  0.04494308],
       [-0.12169714,  0.0463214 , -0.05740227, ...,  0.03618046,
         0.01239075,  0.03830811]], dtype=float32)

In [4]:
data.shape

(10, 384)

((10, 384), (10, 130))

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances

# 1. 加载双视图数据
data = np.load("outputs/multi_view_10.npz",allow_pickle=True)
view1 = data["view_1"]  # textual embedding
view2 = data["view_2"]  # numeric embedding
labels = data["label"]

# 2. 冲突度计算：使用余弦距离
conflict_scores = np.diag(cosine_distances(view1, view2))

# 3. 降维可视化：合并两个视图 + 标签标注
merged = np.concatenate([view1, view2], axis=0)
tsne = TSNE(n_components=2, random_state=42)
proj = tsne.fit_transform(merged)

# 4. 可视化
plt.figure(figsize=(10, 6))
colors = ["red", "blue"]
for i in range(len(view1)):
    plt.scatter(proj[i, 0], proj[i, 1], c="red", label="View 1" if i==0 else "", marker="o")
    plt.scatter(proj[i+len(view1), 0], proj[i+len(view1), 1], c="blue", label="View 2" if i==0 else "", marker="^")
    plt.plot([proj[i, 0], proj[i+len(view1), 0]], [proj[i, 1], proj[i+len(view1), 1]], 'gray', alpha=0.3)

plt.title("T-SNE Visualization of View 1 & View 2 Embeddings")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 5. 冲突度条形图
plt.figure(figsize=(8, 4))
plt.bar(range(len(conflict_scores)), conflict_scores, color="orange")
plt.xlabel("Sample Index")
plt.ylabel("Cosine Distance between Views")
plt.title("View Conflict Score (Cosine Distance)")
plt.grid(True)
plt.tight_layout()
plt.show()

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 384 while Y.shape[1] == 130

In [5]:
from torch.utils.data import Dataset
import numpy as np
class ECMLDataset(Dataset):
    def __init__(self, path_npz):
        data = np.load(path_npz, allow_pickle=True)
        self.X = [data["view_1"], data["view_2"]]
        self.Y = 0  # data["label"]
        self.num_samples = self.X[0].shape[0]
    
    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        x_views = [x[idx].astype(np.float32) for x in self.X]
        y = 0  # int(self.Y[idx])
        return x_views, y

dataset = ECMLDataset("outputs/multi_view_200.npz")
x_views, y = dataset[0] # 第0个数据

# 输出结果
print(type(x_views), len(x_views))

for i in range(5):
    x_views, y = dataset[i]
    print(f"Sample {i}: {x_views[0].shape}, {x_views[1].shape}, {y}")
    print(f"Sample {i}: {x_views[0]}, {x_views[1]}, {y}")

# view1.shape, view2.shape


<class 'list'> 2
Sample 0: (384,), (130,), 0
Sample 0: [-3.68796960e-02 -2.00708266e-02 -2.03424543e-02 -1.57186314e-02
  3.72006074e-02 -2.07259282e-02  6.14560805e-02 -6.52829558e-02
  7.25836307e-02 -2.05737483e-02 -7.76343644e-02  2.45787781e-02
 -6.53959215e-02 -3.25048380e-02  1.82677545e-02 -7.25776795e-03
  7.70300850e-02 -1.37677580e-01 -5.39945252e-02  2.28457563e-02
 -3.28953974e-02  1.86397496e-03 -8.04210976e-02  3.50384153e-02
 -8.47768560e-02  3.19217332e-04 -1.47527400e-02  7.55123328e-03
  6.08310439e-02 -2.22117938e-02 -1.05222734e-02  4.30692323e-02
 -2.45630257e-02  3.63453515e-02 -9.46897492e-02 -6.30303770e-02
 -6.44835830e-03 -3.13473195e-02 -6.19191788e-02  3.05076651e-02
  3.96766253e-02  2.16908287e-02 -2.49292497e-02  9.33288690e-03
 -5.17870523e-02 -7.49211833e-02 -1.02642812e-02  2.72277482e-02
  4.48796041e-02 -6.64813444e-02  4.35952656e-02  2.73912661e-02
 -8.70934781e-03  2.46943142e-02 -1.03739545e-01  5.23390919e-02
 -8.38879272e-02 -1.54034188e-02  6